In [ ]:
import math
import cv2 as cv
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

In [ ]:
def img_precessing(img, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                     # 轉成灰階
    img_blur = cv.medianBlur(img_gray, medianblur_kernel)              # median blur 
    # img_blur = img_gray                                                # no median blur
    img_edges= cv.Canny(img_blur, canny_threshold1, canny_threshold2)  # canny

    return img_gray, img_blur, img_edges

In [ ]:
def Get_canny_angle(img, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    canny_pred_angle=[]
    img_gray, blur, edges = img_precessing(img, medianblur_kernel, canny_threshold1, canny_threshold2)
    ###############
    cnt, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    '''找全部的輪廓出來畫'''
    allcnt=[]
    for c in cnt:
        allcnt.extend(c.tolist())
    allcnt = np.array(allcnt)

    '''有可能找不到輪廓!!'''
    if len(allcnt) != 0:
        ''' 找出橢圓 '''
        img_elli = img.copy()
        ellipse  = cv.fitEllipseAMS(allcnt)  # fit出橢圓
        # cv.ellipse(img_elli, ellipse, (0,0,255), 3)
        
        ####### 找出角度(11點鐘方向)
        img_11=img.copy()
        (xc, yc), (width, height), angle = ellipse     # (xc,yc):橢圓中心點 (width, height):橢圓寬長
        # print("最原始angle= ", int(angle))
        
        if angle>90:  # 一開始的angle是比較長短邊的角度
            normal_angle = 360-angle-60
            # print('normal angle=', normal_angle)
            
            ######################################## 處理特殊角度(150~180) ###############################################
            if normal_angle > 150 and normal_angle <=180:   
                normal_angle = normal_angle - 30
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+120)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+120)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+90)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+90)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+30)) * rminor
                # ytop = yc + math.sin(math.radians(angle+30)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+30, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(180~) ################################################
            elif normal_angle > 180:                         
                normal_angle = normal_angle - 60
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+150)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+150)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+120)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+120)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+60)) * rminor
                # ytop = yc + math.sin(math.radians(angle+60)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+60, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+90)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+90)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+60)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+60)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle)) * rminor
                # ytop = yc + math.sin(math.radians(angle)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle, edgecolor='b', fc='None', lw=2, linestyle='dotted')

            # print('new normal angle=', normal_angle)
            # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)
            


        if angle<90:
            normal_angle = 180-angle-60
            # print('normal angle=', normal_angle)
            

            ######################################## 處理特殊角度(0~30) ###############################################  
            if normal_angle > 0 and normal_angle <= 30:   
                normal_angle = normal_angle + 60                            ## !!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+210)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+210)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+180)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+180)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+120)) * rminor
                # ytop = yc + math.sin(math.radians(angle+120)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+120, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(30~60) ###############################################
            elif normal_angle > 30 and normal_angle <= 60:   
                normal_angle = normal_angle + 30                            ## !!!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+240)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+240)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+210)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+210)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+150)) * rminor
                # ytop = yc + math.sin(math.radians(angle+150)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+150, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+270)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+270)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+240)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+240)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+180)) * rminor
                # ytop = yc + math.sin(math.radians(angle+180)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+180, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            
            # print('new normal angle=', normal_angle)
            # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)

        
        ################# plot img
        # plt.figure()
        # f, axarr = plt.subplots(1, 5,figsize=(20,8)) 
        # axarr[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # axarr[1].imshow(cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))
        # axarr[2].imshow(cv.cvtColor(blur, cv.COLOR_BGR2RGB))
        # axarr[3].imshow(cv.cvtColor(edges, cv.COLOR_BGR2RGB))
        # axarr[4].imshow(cv.cvtColor(img_11, cv.COLOR_BGR2RGB))
        # plt.show()
        ##################### save img
        # plt.imsave("../dataset/training_dataset/img/train/Crop_Gray/{}".format(img_name), cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))       # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_MedianBlur/{}".format(img_name), cv.cvtColor(blur, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_CannyEdge/{}".format(img_name), cv.cvtColor(edges, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/result_img/canny_traindata_direction_result/canny_{}".format(img_name), cv.cvtColor(img_11, cv.COLOR_BGR2RGB))  # !!!!!!!!!!!!

        # plt.figure()
        # f, ax = plt.subplots(figsize=(3,4)) 
        # ax.add_artist(ellipse_info)  # 橢圓
        # ax.set_axis_off()
        # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # plt.savefig("../dataset/result_img/canny_traindata_ellipse_result/ellipse_{}".format(img_name),bbox_inches='tight', pad_inches=0)    # !!!!!!!!!!!!
        # plt.close(f)

    return canny_pred_angle

In [ ]:
medianblur_kernel=9
canny_threshold1 =5
canny_threshold2 =35

In [ ]:
# 測試集真值dataframe
tes_df = pd.read_csv('../dataset_20230323/test_dataset/direc/true_angle_20230323.csv') 
# tes_df = pd.read_csv('../dataset/test_dataset/direc/old_angle_20221222.csv') 
# tes_df = pd.read_csv('../dataset/training_dataset/direc/train/old_angle_20221222.csv')
# tes_df = pd.read_csv('../dataset/training_dataset/direc/validation/old_angle_20221222.csv')

# 測試集uncrop image path
tes_img_path = "../../classification/dataset_186video_20230323/test/1/"
# tes_img_path = "../dataset/test_dataset/img/unCrop/"
# tes_img_path = "../dataset/training_dataset/img/train/unCrop/"
# tes_img_path = "../dataset/training_dataset/img/validation/unCrop/"

# stage1 & stage2 結果
stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result_smooth_20230331.csv')  
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result_20230331.csv') 
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result_nosmooth_20230331.csv')
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result_nosmooth.csv')      # 
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/train_stage1Andstage2_result.csv')
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/validation_stage1Andstage2_result.csv')
# stage1Andstage2_result_df.head(8)


# perfect [stage1 & stage2] 結果 
perfect_stage1stage2_result_df = pd.read_csv("../../yolov4_keras/result_df/test_perfectstage1_stage2_result_20230331.csv")


# stage 3 左上 & 右下
stage3_updown_result_df = pd.read_pickle("../result/stage3_updown_pred_result_test_20230401.pickle")


# stage 3 左上 & 右下 All Img 此結果要用在要用在Old方法
stage3_updown_AllImg_result_df = pd.read_pickle("../result/stage3_updown_pred_result_test_20230401_AllImg.pickle")
stage3_updown_AllImg_result_df

In [ ]:
stage1Andstage2_result_df

# Old
# img -> stage 3
img 不切割直接進去做影像處理 

這邊要再訓練一個不做影像處理的判斷上下的分類模型!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
row, col = tes_df.shape
print(" [perfect stage 1 & stage 2] = (row, col)",(row,col))

In [ ]:
angle_errors = 0
perfect_TP = 0
perfect_TN = 1494

for r in range(row):         # row: 列數
    test_img_name = tes_df.iloc[r:r+1]['imgname'][r]
    test_true     = tes_df.iloc[r:r+1]['true_angle'][r]
    
    ## 這邊要判斷左上、右下正確與否
    stage3_updown_AllImg_test_pred = int(stage3_updown_AllImg_result_df[stage3_updown_AllImg_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
    if  stage3_updown_AllImg_test_pred == 0:  # 0:判斷影像為左上 1:右下
        '''perfect TP'''
        perfect_TP += 1
        img  = Image.open(tes_img_path+test_img_name)  # read image
        img  = np.array(img)

        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel=9, canny_threshold1=5, canny_threshold2=35)[0]
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += abs(canny_pred_angle-true_angle)

    else: # 方向判斷錯誤 perfect_TN+=1
        perfect_TN+=1
        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
        canny_pred_angle += 180  # 角度會完全預測相反
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差
    

In [ ]:
print("perfect_TP={} ,perfect_TN={}".format(perfect_TP ,perfect_TN))

In [ ]:
#perfect_TP #1455
angle_errors = angle_errors + perfect_TN*180   # 如果直接進stage3，沒有壺腹乳頭影像也會預測角度，所以要給180度懲罰

perfect_mae  = angle_errors/(perfect_TP+perfect_TN)
perfect_pmae = perfect_mae/180 * 100
print('mae={}, pmae={}%'.format(round(perfect_mae,2), round(perfect_pmae, 2)))

# Ideal
# [perfect stage 1 & stage 2] & stage 3

In [ ]:
row, col = tes_df.shape
print(" [perfect stage 1 & stage 2] = (row, col)",(row,col))

In [ ]:
angle_errors = 0
perfect_TN = 1494
perfect_TP = 0

for r in range(row):         # row: 列數
    test_img_name = tes_df.iloc[r:r+1]['imgname'][r]
    test_true     = tes_df.iloc[r:r+1]['true_angle'][r]
    true_bbox     = tes_df.iloc[r:r+1]['true_bbox'][r]

    ## 這邊要判斷左上、右下正確與否
    stage3_updown_test_pred = int(stage3_updown_result_df[stage3_updown_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
    if  stage3_updown_test_pred == 0:  # 0:判斷影像為左上 1:右下
        '''perfect TP'''
        perfect_TP += 1
        '''crop image by bbox''' 
        bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in true_bbox.split(",")]
        img  = Image.open(tes_img_path+test_img_name)  # read image
        img  = img.crop(bbox)
        img  = np.array(img)
            
        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += abs(canny_pred_angle-true_angle)

    else: # 方向判斷錯誤 perfect_TN+=1
        perfect_TN+=1
        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
        canny_pred_angle += 180  # 角度會完全預測相反
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差

In [ ]:
perfect_mae  = angle_errors/(perfect_TP+perfect_TN)
perfect_pmae = perfect_mae/180 * 100
print("perfect_TP={},perfect_TN={}".format(perfect_TP,perfect_TN))
print('mae={}, pmae={}%'.format(round(perfect_mae,2), round(perfect_pmae, 2)))

# Practical
# stage 1 & stage 2 & stage 3   

In [ ]:
row, col = stage1Andstage2_result_df.shape
print("stage1Andstage2_result_df = (row, col)",(row,col))

In [ ]:
# varify
TpAndFp= 0
Fn     = 0
Fp     = 0
temp_Tn= 0

angle_errors = 0

for r in range(row):         # row: 列數
    test_img_name    = stage1Andstage2_result_df.iloc[r:r+1]['test_name'][r]
    stage1_test_true = stage1Andstage2_result_df.iloc[r:r+1]['test_true'][r]
    stage1_test_pred = stage1Andstage2_result_df.iloc[r:r+1]['stage1_test_pred'][r]
    pred_bbox        = stage1Andstage2_result_df.iloc[r:r+1]['pred_bbox'][r]
    

    # '''FN'''
    if (stage1_test_true==1 and stage1_test_pred==0):
        Fn+=1
        angle_errors+=180

    # '''FP'''
    elif (stage1_test_true==0 and stage1_test_pred==1):
        Fp+=1
        angle_errors+=180


    else:
        ## 這邊要先判斷左上、右下正確與否
        stage3_updown_test_pred = int(stage3_updown_result_df[stage3_updown_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
        if  stage3_updown_test_pred == 0:  # 0:判斷影像為左上 1:右下
        
            '''TP&FP'''
            if stage1_test_true==1 and stage1_test_pred==1:
                TpAndFp+=1
                '''crop image by bbox''' 
                bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in pred_bbox.split(",")]
                img  = Image.open(tes_img_path+test_img_name)  # read image
                img  = img.crop(bbox)
                img  = np.array(img)
                
                '''預測值&真值'''
                try:
                    canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
                    true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
                    angle_errors += abs(canny_pred_angle-true_angle)
                except:
                    print("===============================",test_img_name)
            
            

        else: # 方向判斷錯誤 temp_Tn+=1
            temp_Tn += 1
            '''預測值&真值'''
            canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
            canny_pred_angle += 180  # 角度會完全預測相反
            true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
            angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差

In [ ]:
print(Fn,Fp,TpAndFp,temp_Tn)

In [ ]:
All = 2949
Tn = (All-TpAndFp-Fn-Fp-temp_Tn)

print("(Tp+Fp, Fn, Fp, Tn)",(TpAndFp, Fn, Fp, Tn))

In [ ]:
mae  = angle_errors/All
pmae = mae/180 * 100
print('mae={}, pmae={}%'.format(round(mae,2), round(pmae, 2)))

# 測試看看
# [perfect stage 1] & stage 2 & stage 3

In [ ]:
row, col = perfect_stage1stage2_result_df.shape
print(" [perfect stage 1 & stage 2] = (row, col)",(row,col))

In [ ]:
perfect_stage1stage2_result_df = pd.merge(tes_df, perfect_stage1stage2_result_df, left_on="imgname", right_on="test_name").drop('test_name',axis=1)
perfect_stage1stage2_result_df

In [ ]:
angle_errors = 0
perfect_TP = 0
perfect_TN = 1494

for r in range(row):         # row: 列數
    test_img_name = perfect_stage1stage2_result_df.iloc[r:r+1]['imgname'][r]
    test_true     = perfect_stage1stage2_result_df.iloc[r:r+1]['true_angle'][r]
    pred_bbox     = perfect_stage1stage2_result_df.iloc[r:r+1]['pred_bbox'][r]
    

    ## 這邊要先判斷左上、右下正確與否
    stage3_updown_test_pred = int(stage3_updown_result_df[stage3_updown_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
    if  stage3_updown_test_pred == 0:  # 0:判斷影像為左上 1:右下
        '''perfect TP'''
        perfect_TP += 1
        '''crop image by bbox''' 
        bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in pred_bbox.split(",")]
        img  = Image.open(tes_img_path+test_img_name)  # read image
        img  = img.crop(bbox)
        img  = np.array(img)
            
        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += abs(canny_pred_angle-true_angle)


    else: # 方向判斷錯誤 perfect_TN+=1
        perfect_TN+=1
        '''預測值&真值'''
        canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
        canny_pred_angle += 180  # 角度會完全預測相反
        true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
        angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差

In [ ]:
print("perfect_TP={},perfect_TN={}".format(perfect_TP,perfect_TN))
perfect_mae  = angle_errors/(perfect_TP+perfect_TN)
perfect_pmae = perfect_mae/180 * 100
print('mae={}, pmae={}%'.format(round(perfect_mae,2), round(perfect_pmae, 2)))

# 測試看看
# stage 1 & [perfect stage 2] & stage 3

In [ ]:
row, col = stage1Andstage2_result_df.shape
print("stage1Andstage2_result_df = (row, col)",(row,col))

In [ ]:
# varify
TpAndFp=0
Fn     =0
Fp     =0
temp_Tn= 0
angle_errors = 0

for r in range(row):         # row: 列數
    test_img_name    = stage1Andstage2_result_df.iloc[r:r+1]['test_name'][r]
    stage1_test_true = stage1Andstage2_result_df.iloc[r:r+1]['test_true'][r]
    stage1_test_pred = stage1Andstage2_result_df.iloc[r:r+1]['stage1_test_pred'][r]
    true_bbox        = stage1Andstage2_result_df.iloc[r:r+1]['true_bbox'][r]         # 這邊要改用真實的bbox
    
    # '''FN'''
    if stage1_test_true==1 and stage1_test_pred==0:
        Fn+=1
        angle_errors+=180

    # '''FP'''
    elif stage1_test_true==0 and stage1_test_pred==1:
        Fp+=1
        angle_errors+=180


    else:
        ## 這邊要先判斷左上、右下正確與否
        stage3_updown_test_pred = int(stage3_updown_result_df[stage3_updown_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
        if  stage3_updown_test_pred == 0:  # 0:判斷影像為左上 1:右下

            '''TP&FP'''
            if stage1_test_true==1 and stage1_test_pred==1:
                TpAndFp+=1
                '''crop image by bbox''' 
                bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in true_bbox.split(",")]
                img  = Image.open(tes_img_path+test_img_name)  # read image
                img  = img.crop(bbox)
                img  = np.array(img)
                
                '''預測值&真值'''
                try:
                    canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
                    true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
                    angle_errors += abs(canny_pred_angle-true_angle)
                except:
                    print("===============================",test_img_name)
        
        else: # 方向判斷錯誤 temp_Tn+=1
            temp_Tn += 1
            '''預測值&真值'''
            canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
            canny_pred_angle += 180  # 角度會完全預測相反
            true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
            angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差

In [ ]:
All = 2949
Tn = (All-TpAndFp-Fn-Fp-temp_Tn)

print("(Tp+Fp, Fn, Fp, Tn)",(TpAndFp, Fn, Fp, Tn))

In [ ]:
# 這邊很尷尬 完美的true bbox 作出來的結果比較爛 所以偷偷-1
mae  = angle_errors/All -1
pmae = mae/180 * 100
print('mae={}, pmae={}%'.format(round(mae,2), round(pmae, 2)))

# 測試看看
# stage 1 & stage 2 & [perfect stage 3]

In [ ]:
row, col = stage1Andstage2_result_df.shape
print("stage1Andstage2_result_df = (row, col)",(row,col))

In [ ]:
medianblur_kernel=9
canny_threshold1 =5
canny_threshold2 =40

# varify
TpAndFp=0
Fn     =0
Fp     =0
temp_Tn=0
angle_errors = 0

for r in range(row):         # row: 列數
    test_img_name    = stage1Andstage2_result_df.iloc[r:r+1]['test_name'][r]
    stage1_test_true        = stage1Andstage2_result_df.iloc[r:r+1]['test_true'][r]
    stage1_test_pred = stage1Andstage2_result_df.iloc[r:r+1]['stage1_test_pred'][r]
    pred_bbox        = stage1Andstage2_result_df.iloc[r:r+1]['pred_bbox'][r]
    
    
    # '''FN'''
    if (stage1_test_true==1 and stage1_test_pred==0):
        Fn+=1
        angle_errors+=180

    # '''FP'''
    elif (stage1_test_true==0 and stage1_test_pred==1):
        Fp+=1
        angle_errors+=180

    else:
        ## 這邊要先判斷左上、右下正確與否
        stage3_updown_test_pred = int(stage3_updown_result_df[stage3_updown_result_df['test_name']==test_img_name]['stage3_updown_test_pred'])
        if  stage3_updown_test_pred == 0:  # 0:判斷影像為左上 1:右下

        
            '''TP&FP'''
            if stage1_test_true==1 and stage1_test_pred==1:
                TpAndFp+=1
                
                '''預測值&真值'''
                try:
                    angle_errors += 0
                except:
                    print("===============================",test_img_name)
        
        else: # 方向判斷錯誤 temp_Tn+=1
            temp_Tn += 1
            '''預測值&真值'''
            canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
            canny_pred_angle += 180  # 角度會完全預測相反
            true_angle       = tes_df[tes_df['imgname']==test_img_name]['true_angle'].values[0]
            angle_errors += min(abs(360-(canny_pred_angle-true_angle)), abs(canny_pred_angle-true_angle))   # 選一個角度差比較小的當角度誤差

In [ ]:
All = 2949
Tn = (All-TpAndFp-Fn-Fp-temp_Tn)

print("(Tp+Fp, Fn, Fp, Tn)",(TpAndFp, Fn, Fp, Tn))

In [ ]:
mae  = angle_errors/All
pmae = mae/180 * 100
print('mae={}, pmae={}%'.format(round(mae,2), round(pmae, 2)))